In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
from skimage.filters import gaussian, sobel, scharr
from skimage.segmentation import active_contour
from skimage.feature import canny
from skimage.exposure import histogram, equalize_hist
from scipy import ndimage as ndi
from skimage.color import label2rgb
from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [23]:
x = [0, 2, 3]
np.nanmean([2, 1, np.mean([])])

1.5

In [70]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, 
                          fours, fulls, ratios, inc): 
    
   
    frratios, flratios = ratios
    #print(frratios, flratios)
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    phighlight = morphology.binary_opening(thresh_img, disk(8))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs =skimage.feature.blob_doh(img, min_sigma =8, max_sigma = 17, threshold = 0.007, num_sigma= 15, overlap=0.6)
        
    ghighlight = gaussian(
        morphology.binary_opening(
             (thresh_img^phighlight)
                           , square(4)), sigma = 0.65)
    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs =skimage.feature.blob_doh(img, min_sigma =9, max_sigma = 16, threshold = 0.005, num_sigma= 15, overlap=0.2)
    

    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
   
    glas_on_glas_dists =np.array([connect((gblobs[i][1], gblobs[i][0]),(g_bond, fg_bond), gblobs[i+1:]) for i in range(ngblobs-1)])
    num_gconnections, num_fgconnections = reducer(glas_on_glas_dists)


    plas_on_plas_dists = np.array([connect((pblobs[i][1], pblobs[i][0]),(p_bond, fp_bond), pblobs[i+1:]) for i in range(npblobs-1)])
    num_pconnections, num_fpconnections = reducer(plas_on_plas_dists)


    num_fconnections = num_fpconnections + num_fgconnections
    #print(num_connections)
    
    outline_sep = scharr(frame)
    markers = np.zeros_like(frame)
    markers[frame < threshold_mean(frame)] = 1
    markers[frame > threshold_triangle(frame)+2] = 2
    segmentation = morphology.watershed(outline_sep, markers)
    segmentation = morphology.opening(ndi.binary_fill_holes(segmentation - 1), disk(3))
    labeled_balls_sep, _ = ndi.label(segmentation)
    regions = measure.regionprops(labeled_balls_sep)

    
    #----------------------filter------------------------- can use actual filter
    if fours < limit and  num_pconnections+num_gconnections ==0 and num_blobs >= 4:
        print('fours', fours)        
        print("plasballs")
        plasareas = areas(regions, pblobs, 0.8) 
        print("glasballs")
        glasareas = areas(regions, gblobs, 1.2)
        if len(glasareas)==nb-npb and len(plasareas)==npb:
            garea = np.nanmean(glasareas)
            plarea = np.nanmean(plasareas)        
            frratios.append(plarea/garea)
            print('frratios', frratios)
            fours += 1
    elif  fulls < limit and num_fconnections ==2 and num_blobs >= 4: 
        fulls += 1
        print('fulls', fulls)     
        imtog = np.copy(frame)
        highs = imtog > threshold_minimum(imtog)
        highs= morphology.opening(highs, disk(8))
        imtog[highs==0]=0
        img = scharr(imtog)
        markers = np.zeros_like(frame)
        markers[imtog < threshold_mean(frame)] = 1
        markers[imtog > threshold_minimum(frame)+2] = 2
        segmentation = morphology.watershed(img, markers)
        segmentation = ndi.binary_fill_holes(morphology.closing(segmentation - 1, disk(5)))
        labeled_balls_tog, _ = ndi.label(segmentation)
        region=measure.regionprops(labeled_balls_tog)
        

        allarea = areafinder(regions)
        print('allarea', allarea)
        plarea = areafinder(region)/npb
        print('plarea', plarea)
        glarea = (allarea-npb*plarea)/(nb-npb)
        print('glarea', glarea)
        flratios.append(plarea/glarea)
        print('flratios', flratios)    
            
    else:
        #print(num_fconnections, 'connections', ' ', num_blobs, 'blobs')
        pass
    #print(ratios)
    return [fours, fulls, ratios]



        
# take largest isodata image region, and return aclassification
def areas(regions, blobs, rang):    
    # mathc blob o blob, collect areas into a list
    if len(regions)==0 or len(blobs)==0:
        return([])
    else: 
        areas = []
        blen = len(blobs)
        regions = [r for r in regions if r.eccentricity <= 0.25]#filter(Regions, key=attrgetter('eccentricity') <= 0.25)
        for b in range(blen):
            blob = blobs[b]
            print(blob)
            for r in range(len(regions)):
                reg = regions[r]
                a = reg.area
                coords = reg.centroid
                print(coords)
                if within(blob[0:2], coords, rang*p_bond):#coords[0] == blob[0] and coords[1]==blob[1]:
                    print('match', a)
                    areas.append(a)
                    regions.remove(reg)
                    break
        return(areas)

def areafinder(allregions):    
    # mathc blob o blob, collect areas into a list
    #print(allregions[0].area)
    aregion = max(allregions, key=attrgetter('area'))
    return(aregion.area)
"""def reducer(connections):
    return sum(connections)  """
def within(pt, origin, rad):
    return (vector_dist(pt, origin) <= rad)
def connect(cv, radii, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    num_f_connections = 0
    norm_radius = 15
    total_separations = 0
    normalized_dists = []
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        total_separations += vector_d
        r1,r2 = radii
        if vector_d <= 2*r1 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_connections += 1
        if vector_d <= 2*r2 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_f_connections += 1
        else:
            pass
        if vector_d <= 4*r1: 
            normalized_dists.append(vector_d/(2*norm_radius))
    return (num_connections, num_f_connections)

def reducer(connections):
    if len(connections)== 0:
        return [0, 0]
    else:
        return(np.sum(connections, axis=0)    )

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

In [71]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/W*half/*_21_a.avi")#glob("./tchakamau/W*half/*_[4,7,8]*.avi") +glob("./tchakamau/W*half/*a.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)

trans_runs = {}
#trans_fil = 10

ave = np.average
nb = 4
npb=2
expected_connections = 5
p_bond = 15*1.35
g_bond = 15*1.15
fp_bond = 15*1.75
fg_bond = 15*1.8
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, g_bond, m_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
limit = 5
frratio = []
flratio = []
ratios = [frratio, flratio]
        
for vidnum in range(len(filenames)):
    print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
    #meta = ffprobe(filenames[vidnum])
    #print(meta)#['@nb_frames'])
    #print(5)
    frames = vreader(filenames[vidnum])
    #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
    #print(frames)



    sides = {"p":[], "g":[], "b":[], "ucf":[]}
    classes = {"p":[], "g":[], "b":[], "ucf":[]}
    transitions = [("p->p",[]),("p->g",[]), ("p->b",[]),
                   ("g->p",[]),("g->g",[]), ("g->b",[]),
                  ("b->p",[]),("b->g",[]), ("b->b",[])]

    transitions = OrderedDict(transitions)
    broken_count = 0
    class_num = 0
    origin = 'ucf'
    last_class = 'ucf'
    last_whole = 0
    fours = 0
    fulls = 0
    fnum = 0
    inc = 20
    for fr in frames:
        frame = fr[:, :, 2]
        org = total_threshold_filter(frame, fours, fulls, ratios, inc)
        fours, fulls, ratios = org
        #print(fours, fulls)
        if fours >= limit and fulls >= limit:
            break
    print(fours, fulls, ratios)

    index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------



['./tchakamau\\Washed15vhalf\\45782_15Vpp_21_a.avi']
Number files:  1
Processing vid 0 : ./tchakamau\Washed15vhalf\45782_15Vpp_21_a.avi
fulls 1
allarea 2790
plarea 669.5
glarea 725.5
flratios [0.9228118538938663]
fulls 2
allarea 2818
plarea 647.0
glarea 762.0
flratios [0.9228118538938663, 0.8490813648293963]
fulls 3
allarea 2829
plarea 659.5
glarea 755.0
flratios [0.9228118538938663, 0.8490813648293963, 0.8735099337748344]
fulls 4
allarea 3290
plarea 683.5
glarea 961.5
flratios [0.9228118538938663, 0.8490813648293963, 0.8735099337748344, 0.7108684347373895]
fulls 5
allarea 3240
plarea 685.0
glarea 935.0
flratios [0.9228118538938663, 0.8490813648293963, 0.8735099337748344, 0.7108684347373895, 0.732620320855615]
fours 0
plasballs
[ 96. 118.  17.]
[ 80. 156.  17.]
glasballs
[ 58. 144.  13.]
[  0. 120.   9.]
fours 0
plasballs
[123.  80.  17.]
[73. 83. 17.]
glasballs
[52. 92. 15.]
[  0. 120.   9.]
fours 0
plasballs
[122.  81.  17.]
(38.0, 118.0)
[71. 83. 17.]
(38.0, 118.0)
glasballs
[ 76. 1

fours 0
plasballs
[ 79. 130.  17.]
[ 34. 147.  17.]
glasballs
[ 64. 151.  13.]
[  0. 118.  16.]
fours 0
plasballs
[ 80. 131.  17.]
[ 34. 145.  17.]
glasballs
[ 65. 148.  13.]
[  0. 117.  16.]
fours 0
plasballs
[ 80. 131.  17.]
[ 32. 142.  17.]
glasballs
[ 65. 146.  13.]
[  1. 118.  16.]
fours 0
plasballs
[ 80. 132.  17.]
[ 32. 141.  17.]
glasballs
[ 63. 142.  13.]
[  0. 119.  16.]
fours 0
plasballs
[ 81. 132.  17.]
(122.0, 131.0)
[ 31. 140.  17.]
(122.0, 131.0)
glasballs
[ 61. 139.  13.]
(122.0, 131.0)
[  0. 119.  16.]
(122.0, 131.0)
fours 0
plasballs
[ 82. 132.  17.]
[ 31. 140.  17.]
glasballs
[ 60. 135.  13.]
[  0. 120.  16.]
fours 0
plasballs
[ 83. 132.  17.]
[ 30. 139.  17.]
glasballs
[ 59. 133.  13.]
[  0. 119.  16.]
fours 0
plasballs
[ 84. 132.  17.]
[ 29. 139.  17.]
glasballs
[ 58. 130.  13.]
[  0. 119.  16.]
fours 0
plasballs
[ 85. 132.  17.]
[ 28. 138.  17.]
glasballs
[ 57. 125.   9.]
[  0. 118.  15.]
fours 0
plasballs
[ 86. 132.  17.]
[ 26. 138.  17.]
glasballs
[ 58. 123.   9